In [1]:
import numpy as np
import pandas as pd
import seaborn as sns  # for heatmaps
import torch
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = DEVICE
torch.cuda.set_device(DEVICE)

In [3]:
torch.cuda.is_available()

True

In [ ]:
assert False

# Factory

## Model

**Input argument**: tensors \
**Output**: tensors 
- AlexNet
- ResNet  
- FrameCNN
- ShallowCNN 

In [5]:
import torch 
from torch import nn



In [8]:
import torch 
from torch import nn
from torchsummary import summary


In [6]:
from models.cnn import create_alexnet,create_resnet18
from training.finetuning import FineTuneCNN

In [7]:
import torch
from torch import Tensor
import torch.nn as nn
import torch.nn.fu
from torchsummary import summary
from models.utils import Lambda

class SmallEncoder(nn.Module):
    def __init__(self,l0,l1,l2,l3):
        """
        3-layer CNN Encoder for CNN-LSTM model

        Arguments:
        l1 (int): number of neuron on the 1st layer
        l2 (int): number of neuron on the 2nd layer
        l3 (int): number of neuron on the 3rd layer
        """
        super(SmallEncoder, self).__init__()
        ### 1st ###
        self.conv1 = nn.Conv2d(l0,l1,kernel_size=4,stride=2)
        self.norm1 = nn.BatchNorm2d(l1) # nn.BatchNorm2d()
        self.actv1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=(2,2))
        ### 2nd ###
        self.conv2 = nn.Conv2d(l1,l2,kernel_size=3,stride=2)
        self.norm2 = nn.BatchNorm2d(l2)
        self.actv2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=(2,2))
        ### 3rd ###
        self.conv3 = nn.Conv2d(l2,l3,kernel_size=2,stride=2)
        self.norm3 = Lambda(lambda x:x)
        self.actv3 = nn.Tanh()
        self.pool3 = nn.MaxPool2d(kernel_size=(1,1))
        self.adapool = nn.AdaptiveAvgPool2d((1,1))
        self.flatten = nn.Flatten()


    def forward(self,X):
        X = self.pool1(self.actv1(self.norm1(self.conv1(X))))
        X = self.pool2(self.actv2(self.norm2(self.conv2(X))))
        X = self.pool3(self.actv3(self.norm3(self.conv3(X))))
        X = self.adapool(X)
        X = self.flatten(X)
        return X

class LSTM(nn.Module):
    def __init__(self,seq_size,feature_size):
        """
        2 layer LSTM model: feature_size --> feature_size

        Arguments:
        seq_size (int): length of the sequence
        feature_size (int): feature size of each interval in the sequence
        """
        super(LSTM, self).__init__()
        self.lstm1 = nn.LSTM(feature_size,feature_size,num_layers=2,bidirectional=True)
        # self.lstm2 = nn.LSTM(feature_size,feature_size)


    def forward(self,X):
        X, (h_0,c_0) = self.lstm1(X)
        # X, (h_0,c_0) = self.lstm2(X, (h_0,c_0))
        X = torch.flatten(X,1)
        return X

    
class DecodeLSTM(nn.Module):
    def __init__(self,seq_size,feature_size,**kwargs):
        """
        2 layer LSTM model: feature_size --> feature_size

        Arguments:
        seq_size (int): length of the sequence
        feature_size (int): feature size of each interval in the sequence
        """
        super(DecodeLSTM, self).__init__()
        self.lstm1 = nn.LSTM(feature_size,200,num_layers=2,bidirectional=True)
#         self.lstm2 = nn.LSTM(400,200,bidirectional=True)


    def forward(self,X):
        X, (h_0,c_0) = self.lstm1(X)
#         X, (h_0,c_0) = self.lstm2(X, (h_0,c_0))
        X = torch.flatten(X,1)
        return X


class CNN_LSTM(nn.Module):

    def __init__(self,n_seq=25,n_channel=1,n_feature=128,n_classes=10):
        super(CNN_LSTM, self).__init__()

        self.n_seq = n_seq
        self.n_feature = n_feature
        self.n_classes = n_classes
        self.den_input = self.n_seq*self.n_feature*2 # 2 for bidirectional
        self.n_channel = n_channel

        self.cnn = SmallEncoder(self.n_channel,32,64,self.n_feature)
        self.lstm = LSTM(self.n_seq,self.n_feature)
        self.dcn = nn.Sequential(nn.Linear(self.den_input,64),
                                 nn.ReLU(),
                                 nn.Linear(64,self.n_classes),
                                 nn.Softmax(1))

    def forward(self,X):
        imgsize = X.shape[2:]
        X = X.view(-1,*imgsize)
        X = self.cnn(X)
        X = X.view(-1,self.n_seq,self.n_feature)
        X = self.lstm(X)
        X = self.dcn(X)
        return X


In [ ]:
hidden_layer = 128 # None, 64, 128, 512  
width = 8 # 1,4

alexnet = lambda : create_alexnet((1,8))
resnet = lambda : create_resnet18((1,8))
lstm = lambda: (LSTM(1600,70),224000)

In [21]:
class Conv1DEmbed(nn.Module):
    """
    1D Convolutional Neural Network, return tensor with size: batch_size,seq_len,embed_size

    Each Block consists: conv(l,k,s) -> batchnorm -> relu -> conv(l,k,s) -> relu
    where l,k,s are number of neurons, kernal size and stride respectively


    """
    def __init__(self,in_channels=70,pooling_size=100,**kwargs):
        """
        Args:
        in_channels (int) - input channel
        pooling_size (int) - size of the last dimension
        layers (list<int>) - number of neurons on each conv layer, length must be equal to 3
        kernel_sizes (list<int>) - kernel_size on each conv layer, length must be equal to 3
        strides (list<int>) - stride on on each conv layer, length must be equal to 3
        """
        super().__init__()
        # parameters
        layers = kwargs.get('layers',[128,256,512,1024])
        kernel_sizes = kwargs.get('kernel_sizes',[32,16,8,4])
        strides = kwargs.get('strides',[4,4,2,2])
        assert len(layers) == len(kernel_sizes) == len(strides)
        l0 = in_channels
        l1,l2,l3,l4 = layers
        k1,k2,k3,k4 = kernel_sizes
        s1,s2,s3,s4 = strides
        self.conv1 = nn.Conv1d(in_channels=l0,out_channels=l1,kernel_size=k1,stride=s1)
        self.conv2 = nn.Conv1d(in_channels=l1,out_channels=l2,kernel_size=k2,stride=s2)
        self.conv3 = nn.Conv1d(in_channels=l2,out_channels=l3,kernel_size=k3,stride=s3)
        self.conv4 = nn.Conv1d(in_channels=l3,out_channels=l4,kernel_size=k4,stride=s4)
        self.norm0 = nn.InstanceNorm1d(l0) # LayerNorm(6400,eps=1e-6)
        self.norm1 = nn.InstanceNorm1d(l1)
        self.norm2 = nn.InstanceNorm1d(l2)
        self.norm3 = nn.InstanceNorm1d(l3)
        self.norm4 = nn.InstanceNorm1d(l4)
        self.pool1 = nn.AdaptiveAvgPool1d(pooling_size)
        

    def forward(self,X):
        X = self.norm0(X)
        X = self.norm1(nn.functional.relu(self.conv1(X)))
        X = self.norm2(nn.functional.relu(self.conv2(X)))
        X = self.norm3(nn.functional.relu(self.conv3(X)))
        X = self.norm4(nn.functional.relu(self.conv4(X)))
        X = self.pool1(X)
        return X.transpose(2,1)

In [49]:
def test_model():
    t = torch.rand(64,70,1600)
#     t = t.permute(0,2,1)
    model = Conv1DEmbed(
        in_channels=70,
        pooling_size=25,
        kernel_sizes=[32,16,8,4],
        strides=[4,4,2,2])
#     builder = lambda: (DecodeLSTM(1600,70),224000)
#     model = FineTuneCNN(builder,model_path=None,hidden_layer=64,n_classes=6)
#     return model(t).shape
    return model

# test_model()
summary(test_model(),input_size=(70,1600),batch_size=64,device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
    InstanceNorm1d-1             [64, 70, 1600]               0
            Conv1d-2             [64, 128, 393]         286,848
    InstanceNorm1d-3             [64, 128, 393]               0
            Conv1d-4              [64, 256, 95]         524,544
    InstanceNorm1d-5              [64, 256, 95]               0
            Conv1d-6              [64, 512, 44]       1,049,088
    InstanceNorm1d-7              [64, 512, 44]               0
            Conv1d-8             [64, 1024, 21]       2,098,176
    InstanceNorm1d-9             [64, 1024, 21]               0
AdaptiveAvgPool1d-10             [64, 1024, 25]               0
Total params: 3,958,656
Trainable params: 3,958,656
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 27.34
Forward/backward pass size (MB): 183.06
Params size (MB): 15.10
Est

## Loss

**Input argument**: tensors \
**Output**: loss 
- Cross Entropy Loss (EXAMPLE)
- Triplet loss  
- NT-Xent
- InfoNCE

##    Train 

**Input argument**: model, dataloader, loss \
**Output**: model 
- Supervised learning
- Autoencoder Pretraining
- Predictive PreTraining 
- Contrastive Pretraining
- Time Contrastive learning
- Contrastive Predictive Coding 

In [1]:
from poutyne import Model,Experiment
from losses import NT_Xent
from models.utils import Classifier
from torch.nn import functional as F

import torch 
from torch import nn

In [ ]:
class Supervised_Learning(object):
    """
    Args:
    encoder_builder (func): callable function of the primary encoder (torch.nn.Module)
    batch_size (int): batch size

    kwargs:
    encoder_builder2 (func): callable function of the secondary encoder (torch.nn.Module)
    temperature (float): temperature of NT-Xent
    optimizer (func): callable function of optimizer (torch.optim.Optimizer)
    supervision (bool): trained with label with Supervised Contrastive Learning (Tian 2020)
    """

    def __init__(self,model,**kwargs):
        # kwargs
        criterion = kwargs.get('criterion',nn.CrossEntropyLoss)
        optimizer = kwargs.get('optimizer',torch.optim.Adam)
        lr = kwargs.get('lr',0.001)
        # overall
        self.model = model
        self.criterion = criterion(**kwargs)
        self.optimizer = optimizer(list(self.model.parameters()), lr=lr)

    def train(self,train_loader,epochs=250,verbose=True,rtn_history=True,device=None):
        """
        Return trained model (and history if rtn_history = True)

        Args:
        train_loader (torch.utils.data.dataloader.DataLoader) - the pair dataset
        epochs (int) - epochs
        verbose (bool) - verbose
        rtn_history (bool) - return both the encoder and history
        device (torch.device) - model to be trained on

        Return
        """
        history = {'loss':[]}
        torch.optim.Optimizer
        if device:
            self.model = self.model.to(device)
            self.criterion = self.criterion.to(device)

        for i in range(epochs):
            if verbose: print(f'Epoch {i+1} ',end='')
            for items in train_loader:

                if device:
                    X,y = [i.to(device) for i in items]

                self.optimizer.zero_grad()
                y_pred = self.model(X)

                # SupConLoss
                if self.supervision:
                    loss = self.criterion(y_pred,y)

                loss.backward()
                self.optimizer.step()

                X = X.cpu()
                y_pred = y_pred.cpu()
                y = y.cpu()
                del X,y_pred,y
                if verbose: print('>',end='')

            loss = loss.tolist()
            history['loss'].append(loss)
            if verbose: print(f' loss: {loss}')

        if device:
            self.model = self.model.cpu()
            self.criterion = self.criterion.cpu()

        if rtn_history:
            return self.model,history
        else:
            return self.model

In [15]:
type(torch.utils.data.DataLoader(torch.rand(1,1)))

torch.utils.data.dataloader.DataLoader

## Validation

**Input argument**: model, dataframe, train \
**Output**: score/scores 
- leave-One-Out Validation
- Cross Validation  
- Cross-Domain Validation
- Sample Efficiency 

## Misc

# Laboratory

In [4]:
import torch 
from torch import nn
